# **Genetic Algorithm demonstration using TSP** 
###### Problem statement - Use genetic algorithms to solve the Travelling Salesperson Problem (TSP) on a large fully connected graph (about 50 nodes)
Created by : Abhishek Singh Dhadwal 


**NOTE - Press Ctrl + F9 to run all code snippets (in Colab)**

TSP - Given a set of cities and distances between every pair of cities, find the shortest way of visiting all the cities exactly once and returning to the starting city.


# Pre-preparation command for colab instance : 

Run to import dependencies into the project.

In [ ]:
import math
import random
from timeit import default_timer as timer # Used for timing data
from tqdm import tqdm
import matplotlib.pyplot as plt

#Input Generation :
We generate the input for our program by creating unique co-ordinates having values ranging from 1 to 1000 (both x and y) via pseudorandom number generators.
We find out out the Euclidean distances between all of them and store them in our matrix.

Distances are generated in this manner in order to satisfy triangle inequalities, allowing our heuristic to perform better.

i/p : numV, number of vertices required

o/p : Tgraph, a numV x numV matrix containing distances between all the points, such that Tgraph[i][j] gives distance from the ith point to the jth point

In [ ]:
def randomIPGen(numV):
  #numV = int(input("Enter the number of vertices required for input graph :"))
  indiceDict = {} # Checks for duplicate indices
  coordlist = [] # Stores randomly generated co-ordinates
  for i in range(numV):
    r1 = random.randint(1, 1000)
    r2 = random.randint(1, 1000)
    if r1 not in indiceDict.keys() :
      while r1 not in indiceDict.keys() :
        r1 = random.randint(1, 1000)
        r2 = random.randint(1, 1000)
        indiceDict[r1] = r2
    coordlist.append([r1,r2])
  Tgraph = [[-1 for i in range(numV)] for j in range(numV)]
  for i in range(numV):
    for j in range(numV):
      if Tgraph[i][j] == -1:
        # Eucledian distance
        Tgraph[i][j] = math.sqrt( ((coordlist[i][0]-coordlist[j][0])**2)+((coordlist[i][1]-coordlist[j][1])**2)) 
        Tgraph[j][i] = Tgraph[i][j]
  print("\nRandomly generated graph of ",numV,"vertices :")
  for i in Tgraph :
    print(i)
  print()
  return Tgraph
#distMatrix = randomIPGen(5)

#Population creation :
A member of the population is defined as a random sampling of all the city indices present, i.e a randomly generated route from city 0, traversing through all the cities, and returning to the city of origin itself.

We run a for loop to generate the required number of input samples

Function: createPopulation

i/p : numCities(number of cities present in the list),numPop(number of members required to be created)

o/p : population(list of lists, with each sub-list containing one randomly generated route)

In [ ]:
import random
def createPopulation(numCities,numPop):
  cityList = [i for i in range(1,numCities)]
  population = []
  for i in range(numPop):
    currSample = [0]
    # Generates a random route as input via sampling
    # Note : For smaller number of inputs identical routes may get generated
    currSample.extend(random.sample(cityList, len(cityList)))
    currSample.append(0)
    population.append(currSample)
  print("Population generated :")
  for i in population :
    print(i)
  return population
#pop = (createPopulation(5,5))

#Fitness function :
We define our fitness function as the inverse of the total distance covered by our route

function : fitness()

i/p : path(a list of indices satisfying the hamiltonian cycle property),distMatrix(data containing the distance matrix)

o/p : fitnessVal (the fitness value created by the route)

In [ ]:
def fitness(path,distMatrix):
  # Corner case, fitness function of a route less than two points can cause a ZeroDivisionError
  if len(path) < 2:
    print('less than two points selected')
    return -1
  pathDist = 0
  # Summing up the distance covered in the path
  for i in range(1,len(path)):
    pathDist += distMatrix[path[i-1]][path[i]]
  #print(pathDist)
  return 1/pathDist
#fitnessSample = fitness(pop[0],distMatrix)
#print("fitness score :" , fitnessSample)

#Selection of population input for crossovers
We use the random roulette method for selection of input.
We add all the fitness function values, then scale them in the scale of 1, by dividing each one by the sum. 
Suppose the values for the paths are A(0.4), B(0.3), C(0.25) and D(0.05). Then we create a random floating-point number in the range [0, 1]. We decide our input in the following manner:

random number 
              
              between 0.00 and 0.40 -> pick A
              
              between 0.40 and 0.70 -> pick B

              between 0.70 and 0.95 -> pick C

              between 0.95 and 1.00 -> pick D

Function crossoverPool()

i/p : pathFitness ( list containing the fitness values of all of the paths),nextgenLen(the amount of participants that shall crossover to the next generation, we aim to keep this length the same as our length before, but may experiment with fraction of outputs)

o/p : pathList (the path pairs selected for crossover)

In [ ]:
def crossoverPool(pathFitness,nextgenLen = None):
  
  # Usual case, as we do not attempt to sieve the number of inputs 
  # Per generation
  if not nextgenLen:
    nextgenLen = len(pathFitness)
  
  # Finding cumulative sum and dividing each term to a scale of 1
  cumulativeSum = sum(pathFitness)
  for i in range(len(pathFitness)):
    pathFitness[i] /= cumulativeSum
  
  # We find cumulative sum till the ith member for each member
  cumulativeSumData = []
  tempSum = 0
  for i in range(len(pathFitness)):
    tempSum += pathFitness[i]
    cumulativeSumData.append(tempSum)

  pool = []
  #print("cumulativeSumData: ",cumulativeSumData)
  #print(nextgenLen)
  
  # The cumulative sum shall act as the ranges for the probability values
  for i in range(nextgenLen):

    randomSelect = random.uniform(0, 1)
    #print("Random value is:",randomSelect)
    if randomSelect >= 0 and randomSelect <= cumulativeSumData[0] :
        #print("between 0 and ",cumulativeSumData[0])
        #print("Hence point :",0, "is selected")
        pool.append(0)
    else :   
      for j in range(1,len(cumulativeSumData)):
        if randomSelect >= cumulativeSumData[j-1] and randomSelect <= cumulativeSumData[j]:
          #print("between",cumulativeSumData[j-1],"and ",cumulativeSumData[j])
          #print("Hence point :",j, "is selected")
          pool.append(j)
          break
  
  return pool
#print(crossoverPool([0.40,0.3,0.25,0.05]))
#print(crossoverPool([0.00329705789659154, 0.003361578831013579, 0.003994012528916378, 0.00329191081571447, 0.0030935943701059796]))

# Crossover function :
We perform ordered crossovers in the following manner - 

1. We select a random subset from the first array
2. We select all the elements not present in the subset and place them in the order stored in the second array

Function crossover()

i/p : parentA, parentB ( the arrays selected for crossing over)

o/p : child (the crossover product)


In [ ]:
def crossover(pA,pB):
  # ADD THE 0TH PATH IN ALL THE PATHS
  parentA = pA.copy()
  parentB = pB.copy()
  parentA = [i for i in parentA if i != 0]
  parentB = [i for i in parentB if i != 0]

  # Ranges of subarray to be selected from parent
  sub1 = random.randint(0,len(parentA)-1)
  sub2 = random.randint(0,len(parentA)-1)
  sub1 = min(sub1,sub2)
  sub2 = max(sub1,sub2)
  
  child = [-1 for i in range(len(parentA))]
  #print(len(parentA))
  #print(parentA[sub1:sub2+1])

  # We place the selected subarray in the child array
  child[sub1:sub2+1] = parentA[sub1:sub2+1]
  
  # We remove the items present in the selected parentA subarray from parentB
  for i in parentA[sub1:sub2+1] :
    parentB.remove(i)
  
  # The rest of the child array is pasted with items from parentB 
  # in the order placed in the parentB array
  child[0:sub1] = parentB[0:sub1]
  child[sub2+1:] = parentB[sub1:]
  
  # Adding 0 to both ends in order to ease path computation
  child.insert(0,0)
  child.append(0)
  return child
#a = crossover([0,1,2,3,4,5,6,7,8,9,0],[0,9,8,7,6,5,4,3,2,1,0])
#print(a)

# Mutation function :
We perform mutation by swapping two random cities in our given output. This is done to prevent our program from getting stuck in a local minima.

The probability of this mutation will be low (i.e about 0.05).

function mutate:

i/p : path (path to be mutated)

o/p : mPath (mutated version of our path)

In [ ]:
def mutate(path):
  # We choose the ranges in order to prevent swapping 0 from any other element
  r1 = random.randint(1,len(path)-2)
  r2 = random.randint(1,len(path)-2)
  #Ensuring different random indices are selected 
  while r1 == r2:
    r2 = random.randint(1,len(path)-2)
  temp = path[r1]
  path[r1] = path[r2]
  path[r2] = temp
  return path
#print(mutate([0,1,2,3,4,5,6,7,8,9,0]))

# Main function
We ask the user to input the following features :

1. The number of vertices required (number of cities)
2. The count of initial population
3. The number of generations for the algorithm
4. The probability of mutation (default taken as 0.05)



In [ ]:
numCities = int(input("Enter the number of cities required in the path: "))
numPop = int(input("Enter the count for initial population: "))
numGen = int(input("Enter the number of generations to run the algorithm: "))
probMutation = float(input("Enter the probability of mutation: "))

'''
numCities = 50
numPop = 50
numGen = 1000
probMutation = 0.05
'''
start = timer()
# Creating distance matrix and initial population
distMatrix = randomIPGen(numCities)
initPop = createPopulation(numCities,numPop)

# Stores best paths and best fitness overall, along with generation
bestPathOverall = []
bestFitnessOverall = [-1,-1]
# Stores best fitness of that gen
bestFGen = []
for currGen in tqdm(range(numGen)):
  fitnessGen = []
  # Getting fitness for all of the participants
  for pop in initPop:
    fitnessGen.append(fitness(pop,distMatrix))
  # Finding the best path in current Generation, i.e path with highest
  # fitness function
  maxFitness = max(fitnessGen)
  bestFGen.append(maxFitness)
  bestfitnessGen = fitnessGen.index(maxFitness)

  # We check if the maximum fitness in our current gen is greater than the
  # Maximum overall fitness, if true we replace current overall maximum 
  # without generation's maximum value
  if maxFitness > bestFitnessOverall[0]:
    bestFitnessOverall[0] = maxFitness
    bestFitnessOverall[1] = currGen
    bestPathOverall = initPop[bestfitnessGen].copy()

  # Selecting population for crossing over to create the next generation
  crossoverSelects = crossoverPool(fitnessGen)
  # Stores the next Generation
  nextGen = []
  
  # Creating next generation
  for i in range(len(crossoverSelects)-1) :
    currChild = crossover(initPop[crossoverSelects[i]],initPop[crossoverSelects[i+1]])  
    # If randomly generated float between 0 and 1 is lower than 
    # Probability of mutation, then mutate 
    if random.uniform(0, 1) < probMutation :
      currChild = mutate(currChild)
    nextGen.append(currChild)

  # Using the first and last element as parents for final child 
  currChild = crossover(initPop[crossoverSelects[0]],initPop[crossoverSelects[-1]])
  if random.uniform(0, 1) < probMutation :
    currChild = mutate(currChild)
  nextGen.append(currChild)
  initPop = nextGen.copy()

# We plot 100 equally spaced fitness maxima in our graph
if numGen > 100:
  interval = numGen // 100
else :
  interval = 1
x = [i for i in range(0,numGen,interval)]
selectedGens = [bestFGen[i] for i in range(0,numGen,interval)] 
plt.plot(x, selectedGens,label = "Fitness values")
plt.plot()
plt.xlabel("Generations, selected in intervals of :"+str(interval))
plt.ylabel("Fitness function values for the given generation")
plt.title("Best fitness value for each generation")
plt.legend()
plt.show()
end = timer()

# Printing output
print()
print("Input features : ")
print("Number of cities : ",numCities)
print("Initial population count : ",numPop," Number of generations : ",numGen, " Probability of mutation : ",probMutation)
print("Best fitness value : ",bestFitnessOverall[0], " Generation of occurence : ", bestFitnessOverall[1]+1)
print("Best path overall : ",bestPathOverall)
print("Time taken to run program : ",end-start," seconds")